In [1]:
using CSV, DataFrames, GLM, StatsModels, StatsBase, LinearAlgebra, Polynomials, BenchmarkTools

In [2]:
filip = CSV.read("filip_data.csv", DataFrame)

x = filip.x
y = filip.y
k = 10
n = length(x)

coef_filip = CSV.read("coeficientes_filip.csv", DataFrame)

,parametro,coeficiente
,String3,Float64
1,B0,-1467.49
2,B1,-2772.18
3,B2,-2316.37
4,B3,-1127.97
5,B4,-354.478
6,B5,-75.1242
7,B6,-10.8753
8,B7,-1.06221
9,B8,-0.067


In [3]:
#Esto viene en la documentación de StatsModels en https://juliastats.org/StatsModels.jl/stable/internals/
# en la sección de Extending @formula syntax

# syntax: best practice to define a _new_ function
poly(x, n) = x^n

# type of model where syntax applies: here this applies to any model type
const POLY_CONTEXT = Any

# struct for behavior
struct PolyTerm{T,D} <: AbstractTerm
    term::T
    deg::D
end

Base.show(io::IO, p::PolyTerm) = print(io, "poly($(p.term), $(p.deg))")

# for `poly` use at run-time (outside @formula), return a schema-less PolyTerm
poly(t::Symbol, d::Int) = PolyTerm(term(t), term(d))

# for `poly` use inside @formula: create a schemaless PolyTerm and apply_schema
function StatsModels.apply_schema(t::FunctionTerm{typeof(poly)},
                                  sch::StatsModels.Schema,
                                  Mod::Type{<:POLY_CONTEXT})
    apply_schema(PolyTerm(t.args_parsed...), sch, Mod)
end

# apply_schema to internal Terms and check for proper types
function StatsModels.apply_schema(t::PolyTerm,
                                  sch::StatsModels.Schema,
                                  Mod::Type{<:POLY_CONTEXT})
    term = apply_schema(t.term, sch, Mod)
    isa(term, ContinuousTerm) ||
        throw(ArgumentError("PolyTerm only works with continuous terms (got $term)"))
    isa(t.deg, ConstantTerm) ||
        throw(ArgumentError("PolyTerm degree must be a number (got $t.deg)"))
    PolyTerm(term, t.deg.n)
end

function StatsModels.modelcols(p::PolyTerm, d::NamedTuple)
    col = modelcols(p.term, d)
    reduce(hcat, [col.^n for n in 1:p.deg])
end

# the basic terms contained within a PolyTerm (for schema extraction)
StatsModels.terms(p::PolyTerm) = terms(p.term)
# names variables from the data that a PolyTerm relies on
StatsModels.termvars(p::PolyTerm) = StatsModels.termvars(p.term)
# number of columns in the matrix this term produces
StatsModels.width(p::PolyTerm) = p.deg

StatsBase.coefnames(p::PolyTerm) = coefnames(p.term) .* "^" .* string.(1:p.deg)

# output

In [73]:
# Función para generar la matriz X que necesitamos para todos los métodos

function generar_X(k) # k es numero de regresores
    # Primero vamos a construir la matriz X 
    n = size(filip, 1) #numero de renglones
    
    X = Array{Float64}(undef, n, k + 1)
    X[:, 1] = ones(n)

    for i = 1:k
        X[:, i + 1] = x.^i
    end
    return X
end
precompile(generar_X, (Int64,))

generar_X (generic function with 1 method)

In [74]:
# Funcion para generar los valores de beta con todos los métodos que encontré 

function generacion_resultados(k)
    
    X = generar_X(k)

    ### Polinomial
    x_pol = Polynomials.fit(x, y, k)

    ### Con QR Pivoted
    F = qr(X, Val(true))
    Q = F.Q
    P = F.P
    R = F.R
    
    ### Método de Mike
    # 1. Resolver QRz = y
    z = Q*R \ y
    # 2. Resuelvo beta = Pz
    x_QR = P*z

    ### Inversa de Moore Penrose
    N = pinv(X)
    x_MP = N*y 
    
    return x_pol, x_QR, x_MP
end
precompile(generacion_resultados, (Int64, ))

generacion_resultados (generic function with 1 method)

In [89]:
function polynomial(k)
    ### Polinomial
    x_pol = Polynomials.fit(x, y, k)
    return x_pol
end
precompile(polynomial, (Int64, ))


function qr_pivoted(k)
    ### Con QR Pivoted
    F = qr(X, Val(true))
    Q = F.Q
    P = F.P
    R = F.R
    
    ### Método de Mike
    # 1. Resolver QRz = y
    z = Q*R \ y
    # 2. Resuelvo beta = Pz
    x_QR = P*z
    
    return x_QR
end
precompile(qr_pivoted, (Int64, ))


function inv_MP(k)
    N = pinv(X)
    x_MP = N*y 
    return x_MP
end
precompile(inv_MP, (Int64,))



inv_MP (generic function with 1 method)

In [ ]:
runs = [1:6;]
i = 1 

In [116]:
# Para guardar los coeficientes
function df_resultados(x_pol, x_QR, x_MP, x_fit)
    
    # Vamos a crear un data frame con los resultados
    degree_df = DataFrame(PolFit = coeffs(x_pol.value), 
                     QRPivot = x_QR.value,
                     MoorePenrose = x_MP.value, 
                    LinearFit = coef(x_fit.value))
    CSV.write("resultados_grado_"*string(k)*".csv", degree_df)
end

# Para guardar los tiempos
function df_tiempos(x_pol, x_QR, x_MP, x_fit)
    
    # Vamos a crear un data frame con los resultados
    tiempos_df = DataFrame(PolFit = x_pol.time, 
                     QRPivot = x_QR.time,
                     MoorePenrose = x_MP.time, 
                     LinearFit = x_fit.time)
    CSV.write("tiempos_grado_"*string(k)*".csv", tiempos_df)
end


df_tiempos (generic function with 2 methods)

In [119]:
k = 1
X = generar_X(k)

x_pol = @timed polynomial(k)
x_QR = @timed qr_pivoted(k)
x_MP = @timed inv_MP(k)

### Con fit que no lo pude hacer dentro de la funcion 
x_fit = @timed lm(@formula(y ~ 1 + poly(x, 1)), filip)

df_resultados(x_pol, x_QR, x_MP, x_fit)
df_tiempos(x_pol, x_QR, x_MP, x_fit)

"tiempos_grado_1.csv"

In [120]:
k = 2
X = generar_X(k)

x_pol = @timed polynomial(k)
x_QR = @timed qr_pivoted(k)
x_MP = @timed inv_MP(k)

### Con fit que no lo pude hacer dentro de la funcion 
x_fit = @timed lm(@formula(y ~ 1 + poly(x, 2)), filip)

df_resultados(x_pol, x_QR, x_MP, x_fit)
df_tiempos(x_pol, x_QR, x_MP, x_fit)


"tiempos_grado_2.csv"

In [121]:
k = 3
X = generar_X(k)

x_pol = @timed polynomial(k)
x_QR = @timed qr_pivoted(k)
x_MP = @timed inv_MP(k)

### Con fit que no lo pude hacer dentro de la funcion 
x_fit = @timed lm(@formula(y ~ 1 + poly(x, 3)), filip)

df_resultados(x_pol, x_QR, x_MP, x_fit)
df_tiempos(x_pol, x_QR, x_MP, x_fit)

"tiempos_grado_3.csv"

In [122]:
k = 4
X = generar_X(k)

x_pol = @timed polynomial(k)
x_QR = @timed qr_pivoted(k)
x_MP = @timed inv_MP(k)

### Con fit que no lo pude hacer dentro de la funcion 
x_fit = @timed lm(@formula(y ~ 1 + poly(x, 4)), filip)

df_resultados(x_pol, x_QR, x_MP, x_fit)
df_tiempos(x_pol, x_QR, x_MP, x_fit)

"tiempos_grado_4.csv"

In [123]:
k = 5
X = generar_X(k)

x_pol = @timed polynomial(k)
x_QR = @timed qr_pivoted(k)
x_MP = @timed inv_MP(k)

### Con fit que no lo pude hacer dentro de la funcion 
x_fit = @timed lm(@formula(y ~ 1 + poly(x, 5)), filip)

df_resultados(x_pol, x_QR, x_MP, x_fit)
df_tiempos(x_pol, x_QR, x_MP, x_fit)

"tiempos_grado_5.csv"

In [124]:
k = 6
X = generar_X(k)

x_pol = @timed polynomial(k)
x_QR = @timed qr_pivoted(k)
x_MP = @timed inv_MP(k)

### Con fit que no lo pude hacer dentro de la funcion 
x_fit = @timed lm(@formula(y ~ 1 + poly(x, 6)), filip)

df_resultados(x_pol, x_QR, x_MP, x_fit)
df_tiempos(x_pol, x_QR, x_MP, x_fit)

"tiempos_grado_6.csv"

In [125]:
k = 7
X = generar_X(k)

x_pol = @timed polynomial(k)
x_QR = @timed qr_pivoted(k)
x_MP = @timed inv_MP(k)

### Con fit que no lo pude hacer dentro de la funcion 
x_fit = @timed lm(@formula(y ~ 1 + poly(x, 7)), filip)

df_resultados(x_pol, x_QR, x_MP, x_fit)
df_tiempos(x_pol, x_QR, x_MP, x_fit)

"tiempos_grado_7.csv"

In [126]:
k = 8
X = generar_X(k)

x_pol = @timed polynomial(k)
x_QR = @timed qr_pivoted(k)
x_MP = @timed inv_MP(k)

### Con fit que no lo pude hacer dentro de la funcion 
x_fit = @timed lm(@formula(y ~ 1 + poly(x, 8)), filip)

df_resultados(x_pol, x_QR, x_MP, x_fit)
df_tiempos(x_pol, x_QR, x_MP, x_fit)

"tiempos_grado_8.csv"

In [127]:
k = 9
X = generar_X(k)

x_pol = @timed polynomial(k)
x_QR = @timed qr_pivoted(k)
x_MP = @timed inv_MP(k)

### Con fit que no lo pude hacer dentro de la funcion 
x_fit = @timed lm(@formula(y ~ 1 + poly(x, 9)), filip)

df_resultados(x_pol, x_QR, x_MP, x_fit)
df_tiempos(x_pol, x_QR, x_MP, x_fit)

"tiempos_grado_9.csv"

In [128]:
k = 10
X = generar_X(k)

x_pol = @timed polynomial(k)
x_QR = @timed qr_pivoted(k)
x_MP = @timed inv_MP(k)

### Con fit que no lo pude hacer dentro de la funcion 
x_fit = @timed lm(@formula(y ~ 1 + poly(x, 10)), filip)

df_resultados(x_pol, x_QR, x_MP, x_fit)
df_tiempos(x_pol, x_QR, x_MP, x_fit)

"tiempos_grado_10.csv"

In [12]:
X_10 = generar_X(10)

82×11 Matrix{Float64}:
 1.0  -6.86012  47.0613  -322.846   2214.76   …      -3.36501e7  2.30844e8
 1.0  -4.32413  18.6981   -80.853    349.619         -5.28553e5  2.28553e6
 1.0  -4.35863  18.9976   -82.8035   360.909         -5.67735e5  2.47454e6
 1.0  -4.35843  18.9959   -82.7922   360.844         -5.67502e5  2.47342e6
 1.0  -6.95585  48.3839  -336.551   2341.0           -3.812e7    2.65157e8
 1.0  -6.66115  44.3709  -295.561   1968.77   …      -2.5819e7   1.71984e8
 1.0  -6.35546  40.3919  -256.709   1631.51          -1.69171e7  1.07516e8
 1.0  -6.1181   37.4312  -229.008   1401.09          -1.20102e7  7.34797e7
 1.0  -7.11515  50.6253  -360.207   2562.92          -4.67364e7  3.32537e8
 1.0  -6.81531  46.4484  -316.56    2157.46          -3.17227e7  2.162e8
 1.0  -6.51999  42.5103  -277.167   1807.13   …      -2.12924e7  1.38826e8
 1.0  -6.20412  38.4911  -238.803   1481.57          -1.36183e7  8.44893e7
 1.0  -5.85387  34.2678  -200.599   1174.28          -8.07215e6  4.72533e7
 ⋮  

In [13]:
sing_values = svd(X_10).S

11-element Vector{Float64}:
      7.196911804503488e9
      4.401508610396736e7
 654533.9743164508
  15214.614835538712
    631.1972849011818
     32.166098045243196
      1.9022357713333182
      0.10394054615153829
      0.004981350609655981
      0.0001755632213724922
      4.0707222722316145e-6

In [14]:
XtX = X_10'*X_10
eigenvalues = eigen(XtX).values

11-element Vector{Float64}:
      5.315116975603194e-10
      4.710082747940292e-7
      0.00041929519790480407
      0.15883624903515486
     50.59401182458721
    987.519265832493
 398409.90132596693
      2.3148450428673536e8
      4.284147235393727e11
      1.9373278047396092e15
      5.1795539521801806e19

In [15]:
eigenvalues_2 = sqrt.(eigenvalues)

11-element Vector{Float64}:
      2.3054537461426533e-5
      0.0006863004260482644
      0.02047669890155159
      0.3985426564812791
      7.11294677504248
     31.42481926491373
    631.1971968616202
  15214.61482544778
 654533.974320182
      4.401508610396678e7
      7.196911804503499e9

In [16]:
# Ahora calculamos el numero de condición de la matriz 

num_cond_julia = cond(X_10)

1.7679692504686805e15

In [17]:
# y ahora a manita
sing_values = sort(sing_values)

num_cond_2 = sing_values[length(sing_values)] / sing_values[1]

1.7679692504686795e15

In [24]:
# y verificamos que sean los mismos

num_cond_julia == num_cond_2

# Bueno, se parecen mucho

false

In [5]:
X = generar_X(10)

82×11 Matrix{Float64}:
 1.0  -6.86012  47.0613  -322.846   2214.76   …      -3.36501e7  2.30844e8
 1.0  -4.32413  18.6981   -80.853    349.619         -5.28553e5  2.28553e6
 1.0  -4.35863  18.9976   -82.8035   360.909         -5.67735e5  2.47454e6
 1.0  -4.35843  18.9959   -82.7922   360.844         -5.67502e5  2.47342e6
 1.0  -6.95585  48.3839  -336.551   2341.0           -3.812e7    2.65157e8
 1.0  -6.66115  44.3709  -295.561   1968.77   …      -2.5819e7   1.71984e8
 1.0  -6.35546  40.3919  -256.709   1631.51          -1.69171e7  1.07516e8
 1.0  -6.1181   37.4312  -229.008   1401.09          -1.20102e7  7.34797e7
 1.0  -7.11515  50.6253  -360.207   2562.92          -4.67364e7  3.32537e8
 1.0  -6.81531  46.4484  -316.56    2157.46          -3.17227e7  2.162e8
 1.0  -6.51999  42.5103  -277.167   1807.13   …      -2.12924e7  1.38826e8
 1.0  -6.20412  38.4911  -238.803   1481.57          -1.36183e7  8.44893e7
 1.0  -5.85387  34.2678  -200.599   1174.28          -8.07215e6  4.72533e7
 ⋮  

In [6]:
rango = rank(X)

10